## 自然言語処理 入門

### 事前準備

コマンドプロンプトを開き、以下のコマンドでMeCabをインストール（for Windows）

pip install mecab

※ MeCabについて: https://taku910.github.io/mecab/

※インストール方法 参考サイト：https://qiita.com/yukinoi/items/990b6933d9f21ba0fb43

### ライブラリのインポート

In [1]:
# 日本語の文章を形態素解析できるライブラリをインポート
import MeCab

### 分かち書き

In [2]:
# 引数の -Owakati は、分かち書きをするための引数
mecab = MeCab.Tagger("-Owakati")

# 分かち書きしたい文章を、引数として入力
text = mecab.parse("私はデータサイエンティストです")

# 出力
print(text)

私 は データ サイエンティスト です 



### 形態素解析

In [3]:
# 引数の -Ochasen は、形態素解析をするための引数
mecab = MeCab.Tagger("-Ochasen")

# 分かち書きしたい文章を、引数として入力
text = mecab.parse("私はデータサイエンティストです")

# 出力
print(text)

私	ワタシ	私	名詞-代名詞-一般		
は	ハ	は	助詞-係助詞		
データ	データ	データ	名詞-一般		
サイエンティスト	サイエンティスト	サイエンティスト	名詞-一般		
です	デス	です	助動詞	特殊・デス	基本形
EOS



In [4]:
# 引数を指定しなくても形態素解析ができる
mecab = MeCab.Tagger()

# 分かち書きしたい文章を、引数として入力
text = mecab.parse("私はデータサイエンティストです")

# 出力
print(text)

私	名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
データ	名詞,一般,*,*,*,*,データ,データ,データ
サイエンティスト	名詞,一般,*,*,*,*,サイエンティスト,サイエンティスト,サイエンティスト
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
EOS



### 単語の出現回数を調べる

In [5]:
import MeCab

# テキストファイルを開く
# 中身は、夏目漱石の"吾輩は猫である"の一節（ちなみに、著作権が切れているので自由に使える）
# 日本語はエンコーディングの指定をしないと読み込めない
with open("neko-1.txt", encoding="utf_8") as f:
    # ファイルの読込み
    data = f.read()  

In [6]:
# 読み込んだデータの確認
data

'吾輩は猫である。名前はまだ無い。どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。'

In [7]:
# 分かち書きした文章を用意
mecab = MeCab.Tagger("-Owakati")
text = mecab.parse(data)

In [8]:
# 格納するために空欄の辞書を作成
words = {}

# split()で分かち書きした文章から単語を取り出す
for word in text.split():
    # 単語をキーとして値に1を足していく。
    # 辞書に単語がない、すなわち初めて辞書に登録するときは0+1になる。
    words[word] = words.get(word, 0) + 1

In [9]:
# リストに取り出して単語の出現回数でソート
# vは出現回数、kは単語を指す
d = [(v, k) for k, v in words.items()]

# 出現回数が少ないものから多いものに並び替え
d.sort()

# 出現回数が多い順に並べる
d.reverse()

In [10]:
# 単語出現回数が多い上位30個を出力
for count, word in d[:30]:
    print(count, word)

17 。
15 た
12 で
11 て
10 の
9 は
9 が
7 という
6 し
4 人間
4 を
4 もの
4 ある
3 顔
3 見
3 書生
3 も
3 に
3 と
3 だ
3 この
3 いる
2 猫
2 時々
2 時
2 掌
2 感じ
2 吾輩
2 何
2 事


### 出現した単語をベクトル化&単語の出現回数をカウント

In [11]:
import MeCab
import pandas as pd

# データの読込み
with open("neko-1.txt", encoding="utf_8") as f:
    data = f.read()

# 読み込んだデータの分かち書き
mecab = MeCab.Tagger("-Owakati")
text = mecab.parse(data)

In [12]:
import numpy as np
# 単語をベクトル化するためのライブラリ
from sklearn.feature_extraction.text import CountVectorizer

# 関数を変数として定義する
count = CountVectorizer()
# array型に変換
docs = np.array([text])
# ベクトル化
vec = count.fit_transform(docs)

In [13]:
# 単語の出現回数をカウント
pd.DataFrame(vec.toarray(), columns=count.get_feature_names())

,あっ,あと,あまりに,ある,あろ,いた事,いとも,いる,いわゆる,から,...,落ちつい,薄暗い,薬缶,装飾,見当,記憶,載せ,逢っ,食う,飲む
0,2,1,1,4,1,1,1,3,1,2,...,1,1,1,1,1,1,1,1,1,1


### TF-IDF

In [14]:
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer

# データの読込み
with open("neko-1.txt", encoding="utf_8") as f:
    data = f.read()

# 読み込んだデータの分かち書き
mecab = MeCab.Tagger("-Owakati")
text = mecab.parse(data)

# 関数を変数として定義する
vec_tfidf = TfidfVectorizer()
# ベクトル化する文字列
doc = np.array([text])
# ベクトル化
vec = vec_tfidf.fit_transform(doc)

In [15]:
# 単語数
print('Vocabulary size: {}'.format(len(vec_tfidf.vocabulary_)))

Vocabulary size: 93


In [16]:
import pandas as pd

# ベクトル化したものを表示
pd.DataFrame(vec.toarray(), columns=vec_tfidf.get_feature_names())

,あっ,あと,あまりに,ある,あろ,いた事,いとも,いる,いわゆる,から,...,落ちつい,薄暗い,薬缶,装飾,見当,記憶,載せ,逢っ,食う,飲む
0,0.129914,0.064957,0.064957,0.259828,0.064957,0.064957,0.064957,0.194871,0.064957,0.129914,...,0.064957,0.064957,0.064957,0.064957,0.064957,0.064957,0.064957,0.064957,0.064957,0.064957
